In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import codecs
import os
import gzip
import sys
from bs4 import BeautifulSoup
from itertools import chain        
from contextlib import ExitStack 
import time
import json
from sklearn.metrics import silhouette_samples
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from multiprocessing.dummy import Lock
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.fasttext import load_facebook_model

In [3]:
vec = {}

In [4]:
with open('titles_stemming.json', 'r', encoding='utf8') as file:
    vec = json.load(file)

In [5]:
sentences = []
for value in vec.values():
    sentences.append(value)

In [6]:
model_fasttext = FastText('ft_freqprune_400K_100K_pq_300.bin')

In [7]:
len(sentences)

28026

Давайте попробуем сделать общий файл с эмеддингами текстов.. не просто так же их парсили

Делаем эмбединги и запоминаем их

Запись в файл стринговых эмбеддингов

Попробуем прочитать эмбеддинги - не забудь перевести в float

In [8]:
with open('texts_embeddings.json', 'r', encoding='utf8') as file:
    texts_embedding_from_json = json.load(file)

In [9]:
for key, val in texts_embedding_from_json.items():
    embedding = np.array(list(map(float, texts_embedding_from_json[key])))
    texts_embedding_from_json[key] = embedding

texts_embedding_from_json

## Фичи

Если фасттекст применять к текстам после стемминга, результат получается лучше почему-то.

Для заголовков:
1) Метки кластеров внутри группы
2) Силуэт для объекта группы

In [14]:
class Import:
    def __init__(self):
        self.doc_to_title = {}
        #self.model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')
        with open('docs_titles.tsv', encoding='utf-8') as f:
            for num_line, line in enumerate(f):
                if num_line == 0:
                    continue
                data = line.strip().split('\t', 1)
                doc_id = int(data[0])
                if len(data) == 1:
                    title = ''
                else:
                    title = data[1]
                self.doc_to_title[doc_id] = title
        self.doc_to_vec = TfidfVectorizer().fit_transform([self.doc_to_title[i] for i in self.doc_to_title.keys()])
        self.top_k_cosine = 5 # k ближайших косинусных расстояний
        self.n_features_jac = 3
        self.extra_features = self.cosine
   
    def cosine(self, group):
        
        def jaccard(group):
            X = np.empty(shape=(group.size, self.n_features_jac), dtype=np.float)
            for i, doc_ID in enumerate(group):
                dist = np.empty(shape=(group.size, ), dtype=np.float)
                vec = set(self.doc_to_title[doc_ID])
                for j, doc_j in enumerate(group):
                    if doc_ID == doc_j:
                        dist[j] = 0
                    else:
                        vec_j = set(self.doc_to_title[doc_j])
                        intersec = len(vec.intersection(vec_j))
                        if intersec > 0:
                            dist[j] = intersec/(len(vec) + len(vec_j) - intersec)
                        else:
                            dist[j] = 0
                X[i] = np.sort(dist)[-self.n_features_jac:]
            return X
    
        n = (self.n_features)//2
        X = np.empty(shape=(group.size, self.n_features), dtype=np.float)
        for i, title in enumerate(pairwise_distances(self.doc_to_vec[group], metric='cosine')):
            X[i, :n] = sorted(title)[1:n + 1]
        X[:, n:] = np.mean(X[:, :n], axis=0)
        X[:, :n] /= X[:, n:]
        if (self.n_features_jac==0):
            return X
        X_jac = jaccard(group)
        return np.hstack((X,X_jac))
                
    
    def make_train(self):
        train_data = pd.read_csv('train_groups.csv', encoding='utf-8')
        traingroups_titledata = {}
        for i in range(len(train_data)):
            new_doc = train_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            target = new_doc['target']
            title = self.doc_to_title[doc_id]
            if doc_group not in traingroups_titledata:
                traingroups_titledata[doc_group] = []
            traingroups_titledata[doc_group].append((doc_id, title, target))
        
        y_train = []
        X_train = []
        groups_train = []
        for new_group in traingroups_titledata:
            docs = traingroups_titledata[new_group]
            for k, (doc_id, title, target_id) in enumerate(docs):
                y_train.append(target_id)
                groups_train.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j, target_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_train.append(sorted(all_dist, reverse=True)[0:25]    )
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        groups_train = np.array(groups_train)
        
        # Класстеризация внутри группы:
        i = 0
        My_X = np.zeros((train_data.shape[0], self.top_k_cosine + 2)) # кол-во новых фичей
        groups = train_data.groupby('group_id')
        rand_index = []
        for group_id, group_indx in groups.groups.items():
            group = train_data.iloc[group_indx]
            docs_id = group.doc_id.values
            # сначала заберем все файлы из группы
            embedded_texts = []
            group_titles = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
            embedded_titles = model_fasttext.wv[group_titles]
            targets = group.target.values
            model_titles = DBSCAN(0.475, metric="cosine", min_samples=7)
            labels_titles = model_titles.fit_predict(embedded_titles)
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.7, metric="cosine", min_samples=5)
            labels_texts = model_texts.fit_predict(embedded_texts)
            # добавим фичи из расстояний
            # получим матрицу из попарных расстояний
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine + 1]
            My_X[i: i + group_indx.size, :self.top_k_cosine] = cos_features
            My_X[i: i + group_indx.size, -2] = labels_titles
            My_X[i: i + group_indx.size, -1] = labels_texts
            #My_X[i: i + group_indx.size, -2] = silhouette
            #My_X[i: i + group_indx.size, :-2] = self.extra_features(group.doc_id)
            i += group_indx.size
        
        
        X_train = np.concatenate((X_train, My_X), axis=1)
        return X_train, y_train, groups_train
    
    def make_test(self):
        test_data = pd.read_csv('test_groups.csv', encoding='utf-8')
        testgroups_titledata = {}
        for i in range(len(test_data)):
            new_doc = test_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            title = self.doc_to_title[doc_id]
            if doc_group not in testgroups_titledata:
                testgroups_titledata[doc_group] = []
            testgroups_titledata[doc_group].append((doc_id, title))
            
        X_test = []
        groups_test = []
        for new_group in testgroups_titledata:
            docs = testgroups_titledata[new_group]
            for k, (doc_id, title) in enumerate(docs):
                groups_test.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_test.append(sorted(all_dist, reverse=True)[0:25])
        X_test = np.array(X_test)
        
        i = 0
        My_X = np.zeros((test_data.shape[0], self.top_k_cosine + 2)) # кол-во новых фичей
        groups = test_data.groupby('group_id')
        for group_id, group_indx in groups.groups.items():
            group = test_data.iloc[group_indx]
            docs_id = group.doc_id.values
            group_titles = []
            embedded_texts = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
            embedded_titles = model_fasttext.wv[group_titles]
            model_titles = DBSCAN(0.475, metric="cosine", min_samples=7)
            labels_titles = model_titles.fit_predict(embedded_titles)
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.7, metric="cosine", min_samples=5)
            labels_texts = model_texts.fit_predict(embedded_texts)
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine + 1]
            My_X[i: i + group_indx.size, :self.top_k_cosine] = cos_features
            My_X[i: i + group_indx.size, -2] = labels_titles
            My_X[i: i + group_indx.size, -1] = labels_texts
            My_X[i: i + group_indx.size, -2] = labels_titles
            My_X[i: i + group_indx.size, -1] = labels_texts
            #My_X[i: i + group_indx.size, -2] = silhouette
            #My_X[i: i + group_indx.size, :-2] = self.extra_features(group.doc_id)
            i += group_indx.size
        
        X_test = np.concatenate((X_test, My_X), axis=1)
        groups_test = np.array(groups_test)
        return X_test, groups_test, test_data['pair_id'].tolist()

In [15]:
importer = Import()

In [16]:
X_train, y_train, groups_train = importer.make_train()

In [17]:
X_test, groups_test, pair_ids = importer.make_test()

## Модельки

In [18]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator
from statistics import mean
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
import json
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import catboost
from sklearn.svm import SVC

In [19]:
class Voiting(BaseEstimator):
    def __init__(self):
        self.clf_e_1 = RandomForestClassifier(max_depth=10)
        self.clf_e_2 = ExtraTreesClassifier(max_depth=15)
        self.clf_e_3 = LogisticRegression()
        self.clf_e_4 = catboost.CatBoostClassifier()
        self.clf_e_5 = MLPClassifier(max_iter=800, hidden_layer_sizes=(10, 10), random_state=42)
        self.clf_e_6 = KNeighborsClassifier(10)

    def fit(self, X, y):
        self.clf_e_1.fit(X, y)
        self.clf_e_2.fit(X, y)
        self.clf_e_3.fit(X, y)
        self.clf_e_4.fit(X, y)
        self.clf_e_5.fit(X, y)
        self.clf_e_6.fit(X, y)

    def predict(self, X):
        self.pred = np.transpose(np.array([self.clf_e_1.predict(X),
                                self.clf_e_2.predict(X),
                                self.clf_e_3.predict(X),
                                self.clf_e_4.predict(X),
                                self.clf_e_5.predict(X),
                                self.clf_e_6.predict(X)]))
        def pr(line):
            ar, count = np.unique(line, return_counts = True)
            return ar[np.argmax(count)]
        predict = np.array([pr(line) for line in self.pred])
        return predict   

In [21]:
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)
X_test_sk = my_sk.transform(X_test)

In [22]:
def feature_generator(X_train, X_test, num_model):
    indices = np.array([])
    length = X_train.shape[1]
    features = np.arange(length)
    np.random.shuffle(features)
    for i in range(num_model):
        ind = np.random.randint(0, length, length)
        ind = np.unique(np.append(ind, features[i * (length // num_model) : (i + 1) * (length // num_model + 1)]))
        yield X_train[:, ind], X_test[:, ind]

In [25]:
X_generator = feature_generator(X_train_sk, X_test_sk, 4) 

In [26]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
DTC = DecisionTreeClassifier()

In [28]:
adaDTC = AdaBoostClassifier(DTC, random_state=7)

In [29]:
ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

In [30]:
kfold = StratifiedKFold(n_splits=10)

In [31]:
X_train_ada, X_test_ada = next(X_generator)

In [32]:
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X_train_ada,y_train)

ada_best = gsadaDTC.best_estimator_

Fitting 10 folds for each of 112 candidates, totalling 1120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   12.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   30.6s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 1120 out of 1120 | elapsed:  2.4min finished


In [33]:
ada_best

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random'),
          learning_rate=1.5, n_estimators=2, random_state=7)

In [34]:
res_ada = ada_best.predict_proba(X_train_ada)[:, 1]

In [35]:
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_ada > th, dtype=int)))

0.4466148428675835
0.4466148428675835
0.9977734896838355
0.9976239976239976
0.9971764006538861
0.9974721189591078
0.9974645786726323


In [37]:
X_train_extra, X_test_extra = next(X_generator)

In [38]:
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsExtC.fit(X_train_extra,y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   29.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  4.2min
[Parallel(n_jobs=4)]: Done 540 out of 540 | elapsed:  5.6min finished


0.6898228464348698

In [39]:
res_extra = ExtC_best.predict_proba(X_train_extra)[:, 1]

In [40]:
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_extra > th, dtype=int)))

0.656619803782745
0.7287468701561942
0.7588859416445624
0.7670512633270045
0.7543075941289088
0.7173231090972465
0.6448382126348229


In [41]:
X_train_res, X_test_res = next(X_generator)

In [42]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train_sk,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   36.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  9.3min
[Parallel(n_jobs=4)]: Done 540 out of 540 | elapsed: 14.7min finished


0.6901521996520792

In [44]:
res_forest = RFC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_forest > th, dtype=int)))

0.6756048387096774
0.7485673352435531
0.7757912859223944
0.7855064027939465
0.7760623905777494
0.7396830924603867
0.6681965959074393


In [45]:
X_train_gbc, X_test_gbc = next(X_generator)

In [48]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train_gbc,y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 10 folds for each of 72 candidates, totalling 720 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   21.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  4.0min
[Parallel(n_jobs=4)]: Done 720 out of 720 | elapsed:  7.3min finished


0.6940792325465183

In [49]:
res_grad = GBC_best.predict_proba(X_train_gbc)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_grad > th, dtype=int)))

0.6420008267879289
0.6978357048905895
0.7287773625200213
0.7337019790454016
0.7180707599555767
0.6833390824422214
0.6015125072716696


In [50]:
kfold = StratifiedKFold(n_splits=5)

In [51]:
X_train_svc, X_test_svc = next(X_generator)

StopIteration: 

In [52]:
### SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsSVMC.fit(X_train_res,y_train) # да, тут res, потому что в RandomForest было обучение на всех фичах

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  5.2min
[Parallel(n_jobs=4)]: Done 140 out of 140 | elapsed: 33.6min finished


0.6823377914145985

In [53]:
res_svc = SVMC_best.predict_proba(X_train_res)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_svc > th, dtype=int)))

0.4552478632478632
0.7403888971352233
0.7281538221041113
0.7201161477657688
0.71088322858088
0.7029669588671612
0.6925989672977625


In [58]:
res_test_ada = ada_best.predict_proba(X_test_ada)[:, 1]

In [59]:
res_test_extra = ExtC_best.predict_proba(X_test_extra)[:, 1]

In [60]:
res_test_forest = RFC_best.predict_proba(X_test_sk)[:, 1]

In [61]:
res_test_gbc = GBC_best.predict_proba(X_test_gbc)[:, 1]

In [62]:
res_test_svc = SVMC_best.predict_proba(X_test_res)[:, 1]

In [54]:
res_pls = (res_ada + res_extra + res_forest + res_grad + res_svc) / 5

In [69]:
f1_score(y_train, np.array(res_pls > 0.37, dtype=int))

0.8083563034963006

In [59]:
from sklearn.ensemble import VotingClassifier

In [60]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best), ('adac',ada_best),('gbc',GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train_sk, y_train)

In [61]:
y_pls = np.array(votingC.predict_proba(X_test_sk)[:, 1] > 0.4, dtype=int)

In [73]:
res_pls = (res_ada + res_extra + res_forest + res_grad) / 4

In [78]:
f1_score(y_train, np.array(res_pls > 0.4, dtype=int))

0.8236308753773177

In [79]:
y_voted_pred = (res_test_ada + res_test_extra + res_test_forest + res_test_gbc) / 4

In [80]:
y_pls = np.array(y_voted_pred > 0.4, dtype=int)

In [81]:
with open("subpobeda_4.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))